# EDA, Pandas, and the Grammar of Data

---

## The basic EDA workflow

Below is a basic checklist for the early stages of exploratory data analysis in Python. While not universally applicable, the rubric covers patterns which recur in several data analysis contexts, so useful to keep it in mind when encountering a new dataset.

The basic workflow (enunciated in this form by Chris Beaumont, the first Head TF of cs109 ever) is as follows:

1. **Build** a Dataframe from the data (ideally, put all data in this object)
2. **Clean** the Dataframe. It should have the following properties:
    - Each row describes a single object
    - Each column describes a property of that object
    - Columns are numeric whenever appropriate
    - Columns contain atomic properties that cannot be further decomposed
    
3. Explore **global properties**. Use histograms, scatter plots, and aggregation functions to summarize the data.
4. Explore **group properties**. Use groupby, queries, and small multiples to compare subsets of the data.

At any such point, you might have to deal with **RELATIONAL** aspects of the data, that is, the combining of data from multiple sources into a dataframe, before you do building, or cleaning, or global analysis, or group analysis.

This process transforms the data into a format which is easier to work with, gives you a basic overview of the data's properties, and likely generates several questions for you to follow-up on in subsequent analysis.

In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

### Building a dataframe

The easiest way to build a dataframe is simply to read in a CSV file. We saw an example of this in lab, and we shall see 2 examples here. We'll also see how we may combine multiple data sources into a larger dataframe.

This example is adapted from: https://github.com/tthibo/SQL-Tutorial

The first 3 lines of the file (`!head -3 data/candidates.txt` on mac/unix) look like this.

```
id|first_name|last_name|middle_name|party
33|Joseph|Biden||D
36|Samuel|Brownback||R
```

In [3]:
dfcand=pd.read_csv("./data/candidates.txt", sep='|')
dfcand.head()

   id   first_name  last_name middle_name party
0  33       Joseph      Biden         NaN     D
1  36       Samuel  Brownback         NaN     R
2  34      Hillary    Clinton          R.     D
3  39  Christopher       Dodd          J.     D
4  26         John    Edwards         NaN     D

A pandas dataframe is a set of columns pasted together into a spreadsheet, as shown in the schematic below, which is taken from the cheatsheet above.  The columns in pandas are called *series* objects.

![](images/pandastruct.png)

All the columns in this dataframe:

In [4]:
dfcand.columns

Index(['id', 'first_name', 'last_name', 'middle_name', 'party'], dtype='object')

And the types of these columns:

In [5]:
dfcand.dtypes

id              int64
first_name     object
last_name      object
middle_name    object
party          object
dtype: object

Access to a particular column can be obtained by treating the column name as an "attribute" of the dataframe:

In [6]:
dfcand.first_name

0          Joseph
1          Samuel
2         Hillary
3     Christopher
4            John
5         Rudolph
6            Mike
7            Mike
8          Duncan
9          Dennis
10           John
11         Barack
12            Ron
13           Bill
14           Mitt
15            Tom
16           Fred
Name: first_name, dtype: object

But Pandas supports a dictionary like access to columns. This is very useful when column names have spaces: Python variables cannot have spaces in them.

In [7]:
dfcand['first_name']

0          Joseph
1          Samuel
2         Hillary
3     Christopher
4            John
5         Rudolph
6            Mike
7            Mike
8          Duncan
9          Dennis
10           John
11         Barack
12            Ron
13           Bill
14           Mitt
15            Tom
16           Fred
Name: first_name, dtype: object

We can also get sub-dataframes by choosing a set of series. We pass a list of the columns we want as "dictionary keys" to the dataframe.

In [8]:
columns_i_want=['first_name', 'last_name']
dfcand[columns_i_want]

     first_name   last_name
0        Joseph       Biden
1        Samuel   Brownback
2       Hillary     Clinton
3   Christopher        Dodd
4          John     Edwards
5       Rudolph    Giuliani
6          Mike      Gravel
7          Mike    Huckabee
8        Duncan      Hunter
9        Dennis    Kucinich
10         John      McCain
11       Barack       Obama
12          Ron        Paul
13         Bill  Richardson
14         Mitt      Romney
15          Tom    Tancredo
16         Fred    Thompson

### Categoricals

Even though `party` is a string, it takes on only a finite set of values, 'D', and 'R'. We can model this:

In [9]:
dfcand.party.unique()

array(['D', 'R'], dtype=object)

In [10]:
dfcand['party'] = dfcand['party'].astype("category")

In [11]:
dfcand.dtypes

id                int64
first_name       object
last_name        object
middle_name      object
party          category
dtype: object

In [12]:
dfcand.head()

   id   first_name  last_name middle_name party
0  33       Joseph      Biden         NaN     D
1  36       Samuel  Brownback         NaN     R
2  34      Hillary    Clinton          R.     D
3  39  Christopher       Dodd          J.     D
4  26         John    Edwards         NaN     D

In [13]:
dfcand.party.describe()

count     17
unique     2
top        R
freq       9
Name: party, dtype: object

In [14]:
dfcand.party.cat.categories

Index(['D', 'R'], dtype='object')

In [15]:
dfcand.party.cat.ordered

False

Keep in mind that this is a relatively new feature of Pandas. You dont need to do this, but might find it useful to keep your types straight. (Using categoricals in machine learning algorithms is more complex and usually involves a process called One Hot Encoding)

### Another piece of data

This is a file of people who have contributed money to candidates:
(`!head -3 data/contributors_with_candidate_id.txt`)

```
id|last_name|first_name|middle_name|street_1|street_2|city|state|zip|amount|date|candidate_id
|Agee|Steven||549 Laurel Branch Road||Floyd|VA|24091|500.00|2007-06-30|16
|Ahrens|Don||4034 Rennellwood Way||Pleasanton|CA|94566|250.00|2007-05-16|16
```

In [16]:
dfcwci=pd.read_csv("./data/contributors_with_candidate_id.txt", sep="|")
dfcwci.head()

   id last_name first_name middle_name                street_1 street_2         city state    zip  amount        date  candidate_id
0 NaN      Agee     Steven         NaN  549 Laurel Branch Road      NaN        Floyd    VA  24091   500.0  2007-06-30            16
1 NaN    Ahrens        Don         NaN    4034 Rennellwood Way      NaN   Pleasanton    CA  94566   250.0  2007-05-16            16
2 NaN    Ahrens        Don         NaN    4034 Rennellwood Way      NaN   Pleasanton    CA  94566    50.0  2007-06-18            16
3 NaN    Ahrens        Don         NaN    4034 Rennellwood Way      NaN   Pleasanton    CA  94566   100.0  2007-06-21            16
4 NaN      Akin    Charles         NaN  10187 Sugar Creek Road      NaN  Bentonville    AR  72712   100.0  2007-06-16            16

### Cleaning Data

most of the techniques you will learn about in Pandas are all about getting data in a form that can be used for further analysis. Cleaning usually means dealing with missing values, transforming types appropriately, and taking care of data integrity. But we'll lump everything required to transform data to a form appropriate for analysis **cleaning**, even if what we are doing is for example, combining multiple data sets, or producing processed data from raw data.

Lets start with some regular cleaning:

In [17]:
# delete 
del dfcwci['id']
dfcwci.head()

  last_name first_name middle_name                street_1 street_2         city state    zip  amount        date  candidate_id
0      Agee     Steven         NaN  549 Laurel Branch Road      NaN        Floyd    VA  24091   500.0  2007-06-30            16
1    Ahrens        Don         NaN    4034 Rennellwood Way      NaN   Pleasanton    CA  94566   250.0  2007-05-16            16
2    Ahrens        Don         NaN    4034 Rennellwood Way      NaN   Pleasanton    CA  94566    50.0  2007-06-18            16
3    Ahrens        Don         NaN    4034 Rennellwood Way      NaN   Pleasanton    CA  94566   100.0  2007-06-21            16
4      Akin    Charles         NaN  10187 Sugar Creek Road      NaN  Bentonville    AR  72712   100.0  2007-06-16            16

We can see the size of our data

In [18]:
dfcwci.shape, dfcand.shape

((175, 11), (17, 5))

We will do some more cleaning soon but let us see the EDA process as sliced from another angle: data transformation, used both for cleaning and for seeing single column and multiple column properties in the data.

## Using SQL: Populating the database

In [19]:
ourschema="""
DROP TABLE IF EXISTS "candidates";
DROP TABLE IF EXISTS "contributors";
CREATE TABLE "candidates" (
    "id" INTEGER PRIMARY KEY  NOT NULL ,
    "first_name" VARCHAR,
    "last_name" VARCHAR,
    "middle_name" VARCHAR,
    "party" VARCHAR NOT NULL
);
CREATE TABLE "contributors" (
    "id" INTEGER PRIMARY KEY  AUTOINCREMENT  NOT NULL,
    "last_name" VARCHAR,
    "first_name" VARCHAR,
    "middle_name" VARCHAR,
    "street_1" VARCHAR,
    "street_2" VARCHAR,
    "city" VARCHAR,
    "state" VARCHAR,
    "zip" VARCHAR,
    "amount" INTEGER,
    "date" DATETIME,
    "candidate_id" INTEGER NOT NULL,
    FOREIGN KEY(candidate_id) REFERENCES candidates(id)
);
"""

### SQLITE

We use sqlite here (and recommend Postgres for production purposes). Still sqlite is great for on-disk large databases which wont fit into memory. 

Its also built into Python, but to use the [command line tool](https://www.sqlite.org/cli.html), I recommend you install it: https://www.sqlite.org/download.html. I also recommend you download and install the sqlite browser: http://sqlitebrowser.org .

Python implements a standard database API over all databases. Its called [DBAPI2](http://cewing.github.io/training.codefellows/lectures/day21/intro_to_dbapi2.html). It works across many SQL databases.

There is an even higher level API available, called [SQLAlchemy](http://www.sqlalchemy.org). While we wont use it here, I thoroughly recommend it, either in its direct relational form, or ORM form. Many things in Pandas use it to interface with databases. Here we'll get away with things by using SQLITE.

In [97]:
from sqlite3 import dbapi2 as sq3
import os
PATHSTART="."
def get_db(dbfile):
    sqlite_db = sq3.connect(os.path.join(PATHSTART, dbfile))
    return sqlite_db

Drop tables if they exist and create them.

In [98]:
def init_db(dbfile, schema):
    """Creates the database tables."""
    db = get_db(dbfile)
    db.cursor().executescript(schema)
    # Commit, otherwise nothing happens.
    db.commit()
    return db

In [99]:
db=init_db("/tmp/cancont.db", ourschema)

In [104]:
!ls -lrt /tmp/cancont.db

-rw-r--r-- 1 kazuki wheel 36864 Jan 19 10:07 /tmp/cancont.db


#### Populating with Pandas!!

In [100]:
# 
dfcand.to_sql("candidates", db, if_exists = "append", index = False)
dfcwci.to_sql("contributors", db, if_exists = "append", index = False)

In [101]:
sel="""
SELECT * FROM candidates;
"""
c=db.cursor().execute(sel)

In [102]:
c.fetchall()

[(16, 'Mike', 'Huckabee', None, 'R'),
 (20, 'Barack', 'Obama', None, 'D'),
 (22, 'Rudolph', 'Giuliani', None, 'R'),
 (24, 'Mike', 'Gravel', None, 'D'),
 (26, 'John', 'Edwards', None, 'D'),
 (29, 'Bill', 'Richardson', None, 'D'),
 (30, 'Duncan', 'Hunter', None, 'R'),
 (31, 'Dennis', 'Kucinich', None, 'D'),
 (32, 'Ron', 'Paul', None, 'R'),
 (33, 'Joseph', 'Biden', None, 'D'),
 (34, 'Hillary', 'Clinton', 'R.', 'D'),
 (35, 'Mitt', 'Romney', None, 'R'),
 (36, 'Samuel', 'Brownback', None, 'R'),
 (37, 'John', 'McCain', None, 'R'),
 (38, 'Tom', 'Tancredo', None, 'R'),
 (39, 'Christopher', 'Dodd', 'J.', 'D'),
 (41, 'Fred', 'Thompson', 'D.', 'R')]

In [26]:
def make_query(sel):
    c=db.cursor().execute(sel)
    return c.fetchall()

In [27]:
make_query("SELECT * FROM candidates;")

[(16, 'Mike', 'Huckabee', None, 'R'),
 (20, 'Barack', 'Obama', None, 'D'),
 (22, 'Rudolph', 'Giuliani', None, 'R'),
 (24, 'Mike', 'Gravel', None, 'D'),
 (26, 'John', 'Edwards', None, 'D'),
 (29, 'Bill', 'Richardson', None, 'D'),
 (30, 'Duncan', 'Hunter', None, 'R'),
 (31, 'Dennis', 'Kucinich', None, 'D'),
 (32, 'Ron', 'Paul', None, 'R'),
 (33, 'Joseph', 'Biden', None, 'D'),
 (34, 'Hillary', 'Clinton', 'R.', 'D'),
 (35, 'Mitt', 'Romney', None, 'R'),
 (36, 'Samuel', 'Brownback', None, 'R'),
 (37, 'John', 'McCain', None, 'R'),
 (38, 'Tom', 'Tancredo', None, 'R'),
 (39, 'Christopher', 'Dodd', 'J.', 'D'),
 (41, 'Fred', 'Thompson', 'D.', 'R')]

## Data transformation: Single Table Verbs

Let us now focus on core data manipulation commands. These are *universal across systems, and by identifying them, we can quickly ask how to do these* when we encounter a new system.

See https://gist.github.com/TomAugspurger/6e052140eaa5fdb6e8c0/ which has a comparison of r/dplyr and pandas. I stole and modified this table from there:

``dplyr`` has a small set of nicely defined verbs, which Hadley Wickham has used to identify core data manipulation commands. Here are listed the closest SQL and Pandas verbs, so we can see the universality of these manipulations.


<table>
  <tr>
    <th><b>VERB</b></th>
    <th><b>dplyr</b></th>
    <th><b>pandas</b></th>
    <th><b>SQL</b></th>
  </tr>
  <tr>
    <td>QUERY/SELECTION</td>
    <td>filter() (and slice())</td>
    <td>query() (and loc[], iloc[])</td>
    <td>SELECT WHERE</td>
  </tr>
  <tr>
    <td>SORT</td>
    <td>arrange()</td>
    <td>sort()</td>
    <td>ORDER BY</td>
  </tr>
  <tr>
    <td>SELECT-COLUMNS/PROJECTION</td>
    <td>select() (and rename())</td>
    <td>[](__getitem__) (and rename())</td>
    <td>SELECT COLUMN</td>
  </tr>
  <tr>
    <td>SELECT-DISTINCT</td>
    <td>distinct()</td>
    <td>unique(),drop_duplicates()</td>
    <td>SELECT DISTINCT COLUMN</td>
  </tr>
  <tr>
    <td>ASSIGN</td>
    <td>mutate() (and transmute())</td>
    <td>assign</td>
    <td>ALTER/UPDATE</td>
  </tr>
  <tr>
    <td>AGGREGATE</td>
    <td>summarise()</td>
    <td>describe(), mean(), max()</td>
    <td>None, AVG(),MAX()</td>
  </tr>
  <tr>
    <td>SAMPLE</td>
    <td>sample_n() and sample_frac()</td>
    <td>sample()</td>
    <td>implementation dep, use RAND()</td>
  </tr>
  <tr>
    <td>GROUP-AGG</td>
    <td>group_by/summarize</td>
    <td>groupby/agg, count, mean</td>
    <td>GROUP BY</td>
  </tr>
  <tr>
    <td>DELETE</td>
    <td>?</td>
    <td>drop/masking</td>
    <td>DELETE/WHERE</td>
  </tr>
</table>


We'll tackle these one by one in Pandas, since these are data manipulations you will do all the time.

### QUERY

In [28]:
# boolean
dfcwci.amount < 400

0      False
1       True
2       True
3       True
4       True
5      False
6      False
7       True
8      False
9      False
10     False
11     False
12      True
13     False
14      True
15     False
16     False
17      True
18      True
19     False
20     False
21     False
22      True
23     False
24     False
25      True
26      True
27      True
28     False
29      True
       ...  
145     True
146     True
147     True
148     True
149     True
150    False
151     True
152     True
153     True
154     True
155    False
156    False
157     True
158     True
159    False
160     True
161     True
162     True
163    False
164    False
165     True
166     True
167     True
168    False
169    False
170     True
171    False
172     True
173    False
174    False
Name: amount, Length: 175, dtype: bool

This gives us Trues and Falses. Such a series is called a **mask**.  A mask  is the basis of filtering. We can do:

In [29]:
dfcwci[dfcwci.amount < 400].head()

  last_name first_name middle_name                      street_1 street_2         city state    zip  amount        date  candidate_id
1    Ahrens        Don         NaN          4034 Rennellwood Way      NaN   Pleasanton    CA  94566   250.0  2007-05-16            16
2    Ahrens        Don         NaN          4034 Rennellwood Way      NaN   Pleasanton    CA  94566    50.0  2007-06-18            16
3    Ahrens        Don         NaN          4034 Rennellwood Way      NaN   Pleasanton    CA  94566   100.0  2007-06-21            16
4      Akin    Charles         NaN        10187 Sugar Creek Road      NaN  Bentonville    AR  72712   100.0  2007-06-16            16
7  Aldridge    Brittni         NaN  808 Capitol Square Place, SW      NaN   Washington    DC  20024   250.0  2007-06-06            16

Notice that the dataframe has been filtered down to only include those contributions with amount < 400. The rows with `False` in the mask have been eliminated, and those with `True` in the mask have been kept.

In [30]:
# true count, true proportion
np.sum(dfcwci.amount < 400), np.mean(dfcwci.amount < 400)

(132, 0.75428571428571434)

Why did that work? The booleans are coerced to integers as below:

In [31]:
1*True, 1*False

(1, 0)

Or directly, in Pandas, which works since the comparison is a pandas Series.

In [32]:
(dfcwci.amount < 400).mean()

0.75428571428571434

You can combine queries. Note that we use Python's `&` operator instead of `and`. This is because we are "Boolean AND"ing masks to get a series of True's And Falses.

In [33]:
dfcwci[(dfcwci.state=='VA') & (dfcwci.amount < 400)]

     last_name first_name middle_name                     street_1 street_2             city state        zip  amount        date  candidate_id
27    Buckheit      Bruce         NaN                8904 KAREN DR      NaN          FAIRFAX    VA  220312731  100.00  2007-09-19            20
77   Ranganath      Anoop         NaN           2507 Willard Drive      NaN  Charlottesville    VA      22903 -100.00  2008-04-21            32
88   Perreault     Louise         NaN    503 Brockridge Hunt Drive      NaN          Hampton    VA      23666  -34.08  2008-04-21            32
145    ABDELLA     THOMAS          M.  4231 MONUMENT WALL WAY #340      NaN          FAIRFAX    VA  220308440   50.00  2007-09-30            35

Here is another way to write the query:

In [34]:
# Throwing the query to SQL as a string
dfcwci.query("state=='VA' & amount < 400")

     last_name first_name middle_name                     street_1 street_2             city state        zip  amount        date  candidate_id
27    Buckheit      Bruce         NaN                8904 KAREN DR      NaN          FAIRFAX    VA  220312731  100.00  2007-09-19            20
77   Ranganath      Anoop         NaN           2507 Willard Drive      NaN  Charlottesville    VA      22903 -100.00  2008-04-21            32
88   Perreault     Louise         NaN    503 Brockridge Hunt Drive      NaN          Hampton    VA      23666  -34.08  2008-04-21            32
145    ABDELLA     THOMAS          M.  4231 MONUMENT WALL WAY #340      NaN          FAIRFAX    VA  220308440   50.00  2007-09-30            35

For cleaning, we might want to use this querying ability

In [35]:
dfcwci[dfcwci.state.isnull()]

    last_name first_name middle_name       street_1 street_2        city state   zip  amount        date  candidate_id
125    BOURNE     TRAVIS         NaN  LAGE KAART 77      NaN  BRASSCHATT   NaN  2930  -500.0  2008-11-20            35

Or the opposite, which is probably more useful in making the selection:

In [36]:
dfcwci[dfcwci.state.notnull()].head()

  last_name first_name middle_name                street_1 street_2         city state    zip  amount        date  candidate_id
0      Agee     Steven         NaN  549 Laurel Branch Road      NaN        Floyd    VA  24091   500.0  2007-06-30            16
1    Ahrens        Don         NaN    4034 Rennellwood Way      NaN   Pleasanton    CA  94566   250.0  2007-05-16            16
2    Ahrens        Don         NaN    4034 Rennellwood Way      NaN   Pleasanton    CA  94566    50.0  2007-06-18            16
3    Ahrens        Don         NaN    4034 Rennellwood Way      NaN   Pleasanton    CA  94566   100.0  2007-06-21            16
4      Akin    Charles         NaN  10187 Sugar Creek Road      NaN  Bentonville    AR  72712   100.0  2007-06-16            16

For categoricals you can use `isin`. You can use Boolean not on the mask to implement not in.

In [37]:
dfcwci[dfcwci.state.isin(['VA','WA'])].head(10)

     last_name first_name middle_name                      street_1 street_2             city state        zip  amount        date  candidate_id
0         Agee     Steven         NaN        549 Laurel Branch Road      NaN            Floyd    VA      24091  500.00  2007-06-30            16
27    Buckheit      Bruce         NaN                 8904 KAREN DR      NaN          FAIRFAX    VA  220312731  100.00  2007-09-19            20
62       BURKE    SUZANNE          M.                 3401 EVANSTON      NaN          SEATTLE    WA  981038677 -700.00  2008-03-05            22
77   Ranganath      Anoop         NaN            2507 Willard Drive      NaN  Charlottesville    VA      22903 -100.00  2008-04-21            32
88   Perreault     Louise         NaN     503 Brockridge Hunt Drive      NaN          Hampton    VA      23666  -34.08  2008-04-21            32
100   Aaronson    Rebecca         NaN  2000 Village Green Dr Apt 12      NaN       Mill Creek    WA  980125787  100.00  2008-02-08

And you can chain queries thus.

In [38]:
dfcwci.query("10 <= amount <= 50").head(10)

    last_name first_name middle_name                    street_1 street_2           city state        zip  amount        date  candidate_id
2      Ahrens        Don         NaN        4034 Rennellwood Way      NaN     Pleasanton    CA      94566    50.0  2007-06-18            16
18      Ardle    William         NaN           412 Dakota Avenue      NaN    Springfield    OH      45504    50.0  2007-06-28            16
25    Buckler      Steve         NaN             24351 Armada Dr      NaN     Dana Point    CA  926291306    50.0  2007-07-30            20
26    Buckler      Steve         NaN             24351 Armada Dr      NaN     Dana Point    CA  926291306    25.0  2007-08-16            20
34       Buck    Barbara         NaN            1780 NE 138th St      NaN    North Miami    FL  331811316    50.0  2007-09-13            20
35       Buck    Barbara         NaN            1780 NE 138th St      NaN    North Miami    FL  331811316    50.0  2007-07-19            20
38   Buchanek  Eliza

#### In Database land

In [39]:
cont_cols = [e[1] for e in make_query("PRAGMA table_info(contributors);")]
cont_cols

['id',
 'last_name',
 'first_name',
 'middle_name',
 'street_1',
 'street_2',
 'city',
 'state',
 'zip',
 'amount',
 'date',
 'candidate_id']

In [40]:
def make_frame(list_of_tuples, legend=cont_cols):
    framelist=[]
    for i, cname in enumerate(legend):
        framelist.append((cname,[e[i] for e in list_of_tuples]))
    return pd.DataFrame.from_items(framelist)

In [41]:
out=make_query("SELECT * FROM contributors WHERE state='VA' AND amount < 400;")
print(out)
make_frame(out)

[(28, 'Buckheit', 'Bruce', None, '8904 KAREN DR', None, 'FAIRFAX', 'VA', '220312731', 100, '2007-09-19', 20), (78, 'Ranganath', 'Anoop', None, '2507 Willard Drive', None, 'Charlottesville', 'VA', '22903', -100, '2008-04-21', 32), (89, 'Perreault', 'Louise', None, '503 Brockridge Hunt Drive', None, 'Hampton', 'VA', '23666', -34.08, '2008-04-21', 32), (146, 'ABDELLA', 'THOMAS', 'M.', '4231 MONUMENT WALL WAY #340', None, 'FAIRFAX', 'VA', '220308440', 50, '2007-09-30', 35)]


    id  last_name first_name middle_name                     street_1 street_2             city state        zip  amount        date  candidate_id
0   28   Buckheit      Bruce        None                8904 KAREN DR     None          FAIRFAX    VA  220312731  100.00  2007-09-19            20
1   78  Ranganath      Anoop        None           2507 Willard Drive     None  Charlottesville    VA      22903 -100.00  2008-04-21            32
2   89  Perreault     Louise        None    503 Brockridge Hunt Drive     None          Hampton    VA      23666  -34.08  2008-04-21            32
3  146    ABDELLA     THOMAS          M.  4231 MONUMENT WALL WAY #340     None          FAIRFAX    VA  220308440   50.00  2007-09-30            35

### SORT

In [42]:
dfcwci.sort_values(by="amount").head(10)

   last_name   first_name middle_name                      street_1 street_2                city state        zip  amount        date  candidate_id
90     Kazor  Christopher           M           707 Spindletree ave      NaN          Naperville    IL      60565 -2592.0  2008-04-21            32
72     BRUNO         JOHN         NaN  10136 WINDERMERE CHASE BLVD.      NaN               GOTHA    FL  347344707 -2300.0  2008-03-06            22
64     BURKE       DONALD          J.                     12 LOMPOC      NaN  RANCHO SANTA MARGA    CA  926881817 -2300.0  2008-03-11            22
73     BRUNO        IRENE         NaN  10136 WINDERMERE CHASE BLVD.      NaN               GOTHA    FL  347344707 -2300.0  2008-03-06            22
74     BROWN      TIMOTHY          J.           26826 MARLOWE COURT      NaN     STEVENSON RANCH    CA  913811020 -2300.0  2008-03-06            22
58    BURTON        GLENN          M.         4404 CHARLESTON COURT      NaN               TAMPA    FL  33609262

In [43]:
out=make_query("SELECT * FROM contributors ORDER BY amount;")
make_frame(out).head(10)

   id last_name   first_name middle_name                   street_1 street_2                city state        zip  amount        date  candidate_id
0  91     Kazor  Christopher           M        707 Spindletree ave     None          Naperville    IL      60565 -2592.0  2008-04-21            32
1  30    Buckel        Linda        None              PO Box 683130     None           Park City    UT  840683130 -2300.0  2007-08-14            20
2  52  BYINGTON     MARGARET          E.  2633 MIDDLEBORO LANE N.E.     None        GRAND RAPIDS    MI  495061254 -2300.0  2008-03-03            22
3  53     BYERS          BOB          A.       13170 TELFAIR AVENUE     None              SYLMAR    CA  913423573 -2300.0  2008-03-07            22
4  55      BUSH      KRYSTIE        None             P.O. BOX 61046     None              DENVER    CO  802061046 -2300.0  2008-03-06            22
5  56      BUSH         ERIC        None             P.O. BOX 61046     None              DENVER    CO  80206104

In [44]:
dfcwci[dfcwci.amount < 0]

      last_name   first_name middle_name                      street_1          street_2                city state        zip   amount        date  candidate_id
29       Buckel        Linda         NaN                 PO Box 683130               NaN           Park City    UT  840683130 -2300.00  2007-08-14            20
40     Buchanan         John         NaN               2025 NW 29th Rd               NaN          Boca Raton    FL  334316303  -500.00  2007-09-24            20
43     Buchanan         John         NaN               2025 NW 29th Rd               NaN          Boca Raton    FL  334316303  -700.00  2007-08-28            20
50        BYNUM      HERBERT         NaN            332 SUNNYSIDE ROAD               NaN               TAMPA    FL  336177249  -500.00  2008-03-10            22
51     BYINGTON     MARGARET          E.     2633 MIDDLEBORO LANE N.E.               NaN        GRAND RAPIDS    MI  495061254 -2300.00  2008-03-03            22
52        BYERS          BOB      

In [45]:
dfcwci.sort_values(by="amount", ascending=False).head(10)

      last_name first_name middle_name                 street_1 street_2           city state        zip  amount        date  candidate_id
30       Buckel      Linda         NaN            PO Box 683130      NaN      Park City    UT  840683130  4600.0  2007-08-14            20
159       ABATE      MARIA       ELENA  1291 NIGHTINGALE AVENUE      NaN  MIAMI SPRINGS    FL  331663832  2600.0  2008-01-25            37
15      Anthony       John         NaN    211 Long Island Drive      NaN    Hot Springs    AR      71913  2300.0  2007-06-12            16
33         Buck     Blaine           M             45 Eaton Ave      NaN         Camden    ME   48431752  2300.0  2007-09-30            20
28       Buckel      Linda         NaN            PO Box 683130      NaN      Park City    UT  840683130  2300.0  2007-08-14            20
21        Baker      David         NaN   2550 Adamsbrooke Drive      NaN         Conway    AR      72034  2300.0  2007-04-11            16
13        Altes       R.D. 

In [46]:
out=make_query("SELECT * FROM contributors ORDER BY amount DESC;")
make_frame(out).head(10)

    id   last_name first_name middle_name                 street_1 street_2           city state        zip  amount        date  candidate_id
0   31      Buckel      Linda        None            PO Box 683130     None      Park City    UT  840683130  4600.0  2007-08-14            20
1  160       ABATE      MARIA       ELENA  1291 NIGHTINGALE AVENUE     None  MIAMI SPRINGS    FL  331663832  2600.0  2008-01-25            37
2   14       Altes       R.D.        None          8600 Moody Road     None     Fort Smith    AR      72903  2300.0  2007-06-21            16
3   16     Anthony       John        None    211 Long Island Drive     None    Hot Springs    AR      71913  2300.0  2007-06-12            16
4   22       Baker      David        None   2550 Adamsbrooke Drive     None         Conway    AR      72034  2300.0  2007-04-11            16
5   29      Buckel      Linda        None            PO Box 683130     None      Park City    UT  840683130  2300.0  2007-08-14            20
6   34

### SELECT-COLUMNS

In [47]:
dfcwci[['first_name', 'amount']].head(10)

  first_name  amount
0     Steven   500.0
1        Don   250.0
2        Don    50.0
3        Don   100.0
4    Charles   100.0
5       Mike  1500.0
6    Rebecca   500.0
7    Brittni   250.0
8    John D.  1000.0
9    John D.  1300.0

In [48]:
out=make_query("SELECT first_name, amount FROM contributors;")
make_frame(out,['first_name', 'amount']).head(10)

  first_name  amount
0     Steven   500.0
1        Don   250.0
2        Don    50.0
3        Don   100.0
4    Charles   100.0
5       Mike  1500.0
6    Rebecca   500.0
7    Brittni   250.0
8    John D.  1000.0
9    John D.  1300.0

### SELECT-DISTINCT

Selecting a distinct set is useful for cleaning. Here, we might wish to focus on contributors rather than contributions and see how many distinct contributors we have. Of-course we might be wrong, some people have identical names. 

In [49]:
dfcwci[['last_name','first_name']].count()

last_name     175
first_name    175
dtype: int64

In [50]:
dfcwci[['last_name','first_name']].drop_duplicates().count()

last_name     126
first_name    126
dtype: int64

In [51]:
dfcwci[['last_name','first_name']].drop_duplicates().head(10)

   last_name first_name
0       Agee     Steven
1     Ahrens        Don
4       Akin    Charles
5       Akin       Mike
6       Akin    Rebecca
7   Aldridge    Brittni
8      Allen    John D.
10   Allison    John W.
11   Allison    Rebecca
13     Altes       R.D.

In [52]:
out=make_query("SELECT DISTINCT last_name, first_name FROM contributors;")
make_frame(out,['last_name', 'first_name']).head(10)

  last_name first_name
0      Agee     Steven
1    Ahrens        Don
2      Akin    Charles
3      Akin       Mike
4      Akin    Rebecca
5  Aldridge    Brittni
6     Allen    John D.
7   Allison    John W.
8   Allison    Rebecca
9     Altes       R.D.

### ASSIGN

Assignment to a new column is easy.

In [53]:
dfcwci['name']=dfcwci['last_name']+", "+dfcwci['first_name']
dfcwci.head(10)

  last_name first_name middle_name                      street_1 street_2           city state    zip  amount        date  candidate_id               name
0      Agee     Steven         NaN        549 Laurel Branch Road      NaN          Floyd    VA  24091   500.0  2007-06-30            16       Agee, Steven
1    Ahrens        Don         NaN          4034 Rennellwood Way      NaN     Pleasanton    CA  94566   250.0  2007-05-16            16        Ahrens, Don
2    Ahrens        Don         NaN          4034 Rennellwood Way      NaN     Pleasanton    CA  94566    50.0  2007-06-18            16        Ahrens, Don
3    Ahrens        Don         NaN          4034 Rennellwood Way      NaN     Pleasanton    CA  94566   100.0  2007-06-21            16        Ahrens, Don
4      Akin    Charles         NaN        10187 Sugar Creek Road      NaN    Bentonville    AR  72712   100.0  2007-06-16            16      Akin, Charles
5      Akin       Mike         NaN              181 Baywood Lane      

In [54]:
dfcwci.assign(ucname=dfcwci.last_name+":"+dfcwci.first_name).head(10)

  last_name first_name middle_name                      street_1 street_2           city state    zip  amount        date  candidate_id               name            ucname
0      Agee     Steven         NaN        549 Laurel Branch Road      NaN          Floyd    VA  24091   500.0  2007-06-30            16       Agee, Steven       Agee:Steven
1    Ahrens        Don         NaN          4034 Rennellwood Way      NaN     Pleasanton    CA  94566   250.0  2007-05-16            16        Ahrens, Don        Ahrens:Don
2    Ahrens        Don         NaN          4034 Rennellwood Way      NaN     Pleasanton    CA  94566    50.0  2007-06-18            16        Ahrens, Don        Ahrens:Don
3    Ahrens        Don         NaN          4034 Rennellwood Way      NaN     Pleasanton    CA  94566   100.0  2007-06-21            16        Ahrens, Don        Ahrens:Don
4      Akin    Charles         NaN        10187 Sugar Creek Road      NaN    Bentonville    AR  72712   100.0  2007-06-16            16

Will the above command actually change `dfcwci`?. No, it produces a fresh dataframe.

#### What if we wanted to change an existing assignment?

In [55]:
dfcwci[dfcwci.state=='VA']

     last_name first_name middle_name                     street_1 street_2             city state        zip  amount        date  candidate_id               name
0         Agee     Steven         NaN       549 Laurel Branch Road      NaN            Floyd    VA      24091  500.00  2007-06-30            16       Agee, Steven
27    Buckheit      Bruce         NaN                8904 KAREN DR      NaN          FAIRFAX    VA  220312731  100.00  2007-09-19            20    Buckheit, Bruce
77   Ranganath      Anoop         NaN           2507 Willard Drive      NaN  Charlottesville    VA      22903 -100.00  2008-04-21            32   Ranganath, Anoop
88   Perreault     Louise         NaN    503 Brockridge Hunt Drive      NaN          Hampton    VA      23666  -34.08  2008-04-21            32  Perreault, Louise
145    ABDELLA     THOMAS          M.  4231 MONUMENT WALL WAY #340      NaN          FAIRFAX    VA  220308440   50.00  2007-09-30            35    ABDELLA, THOMAS

In [56]:
dfcwci.loc[dfcwci.state=='VA', 'name']

0           Agee, Steven
27       Buckheit, Bruce
77      Ranganath, Anoop
88     Perreault, Louise
145      ABDELLA, THOMAS
Name: name, dtype: object

In [57]:
dfcwci.loc[dfcwci.state=='VA', 'name']="junk"

In [58]:
dfcwci.query("state=='VA'")

     last_name first_name middle_name                     street_1 street_2             city state        zip  amount        date  candidate_id  name
0         Agee     Steven         NaN       549 Laurel Branch Road      NaN            Floyd    VA      24091  500.00  2007-06-30            16  junk
27    Buckheit      Bruce         NaN                8904 KAREN DR      NaN          FAIRFAX    VA  220312731  100.00  2007-09-19            20  junk
77   Ranganath      Anoop         NaN           2507 Willard Drive      NaN  Charlottesville    VA      22903 -100.00  2008-04-21            32  junk
88   Perreault     Louise         NaN    503 Brockridge Hunt Drive      NaN          Hampton    VA      23666  -34.08  2008-04-21            32  junk
145    ABDELLA     THOMAS          M.  4231 MONUMENT WALL WAY #340      NaN          FAIRFAX    VA  220308440   50.00  2007-09-30            35  junk

### Drop-Column

Real simple:

In [59]:
del dfcwci['name']

#### No DROP COLUMN in SQLITE

Its available in other databases. Here you must just re-create your database, or no about this gotcha from the start.

### AGGREGATE

In [60]:
dfcwci.describe()

                zip       amount  candidate_id
count  1.750000e+02   175.000000    175.000000
mean   3.780014e+08     3.418114     28.000000
std    3.628278e+08  1028.418999      7.823484
min    2.474000e+03 -2592.000000     16.000000
25%    9.336700e+04  -175.000000     20.000000
50%    3.233313e+08   100.000000     32.000000
75%    7.816946e+08   300.000000     35.000000
max    9.951532e+08  4600.000000     37.000000

In [61]:
dfcwci.amount.max()

4600.0

In [62]:
dfcwci[dfcwci.amount==dfcwci.amount.max()]

   last_name first_name middle_name       street_1 street_2       city state        zip  amount        date  candidate_id
30    Buckel      Linda         NaN  PO Box 683130      NaN  Park City    UT  840683130  4600.0  2007-08-14            20

In [63]:
dfcwci[dfcwci.amount > dfcwci.amount.max() - 2300]

    last_name first_name middle_name                 street_1 street_2           city state        zip  amount        date  candidate_id
30     Buckel      Linda         NaN            PO Box 683130      NaN      Park City    UT  840683130  4600.0  2007-08-14            20
159     ABATE      MARIA       ELENA  1291 NIGHTINGALE AVENUE      NaN  MIAMI SPRINGS    FL  331663832  2600.0  2008-01-25            37

In [64]:
out=make_query("SELECT * FROM contributors WHERE amount > (select (MAX(amount) - 2300) FROM contributors);")
make_frame(out)

    id last_name first_name middle_name                 street_1 street_2           city state        zip  amount        date  candidate_id
0   31    Buckel      Linda        None            PO Box 683130     None      Park City    UT  840683130    4600  2007-08-14            20
1  160     ABATE      MARIA       ELENA  1291 NIGHTINGALE AVENUE     None  MIAMI SPRINGS    FL  331663832    2600  2008-01-25            37

Aso `MIN`, `SUM`, `AVG`.

In [65]:
dfcwci.amount.mean()

3.4181142857142808

In [66]:
out=make_query("SELECT AVG(amount) FROM contributors;")
out

[(3.418114285714276,)]

## Grouping using Pandas and split-apply-combine

In [67]:
grouped_by_state = dfcwci.groupby("state")
grouped_by_state

In [68]:
grouped_by_state.amount

How do we get access to these? Standard pandas functions distribute over the `groupby`, going one by one over the sub-dataframes or sub-series.  This is an example of a paradigm called split-apply-combine.

### GROUP-AGG

The fourth part of the EDA rubric is to look at properties of the sub-dataframes you get when you make groups. (We'll talk about the graphical aspects of this later). For instance, you might group contributions by state:

In [69]:
dfcwci.groupby("state").describe()

      amount                                                                            candidate_id                                                         zip                                                                                               
       count         mean          std      min        25%       50%        75%     max        count       mean        std   min    25%   50%    75%   max count          mean           std          min           25%          50%           75%          max
state                                                                                                                                                                                                                                                          
AK       3.0   403.333333   100.166528   300.00   355.0000   410.000   455.0000   500.0          3.0  37.000000   0.000000  37.0  37.00  37.0  37.00  37.0   3.0  9.951532e+08  0.000000e+00  995153207.0  9.951532e+08  995153207.0  9.

In [70]:
dfcwci.groupby("state").sum()

               zip    amount  candidate_id
state                                     
AK      2985459621   1210.00           111
AR          864790  14200.00           192
AZ       860011121    120.00            37
CA     14736360720  -5013.73           600
CO      2405477834  -5823.00           111
CT        68901376   2300.00            35
DC       800341853  -1549.91           102
FL      8970626520  -4050.00           803
IA           50266    250.00            16
ID           83648   -261.00            32
IL      3042068689  -5586.80           175
KS           66215   -330.00            32
KY       402597029   -200.00            22
LA      1406043327   1300.00            74
MA       123026638    -83.00           208
MD       416287617    300.00            55
ME       165647170   2520.00           122
MI      2426973485  -1265.00           164
MN      1102338918    322.00           100
MO           64111    100.00            20
NC           27502    500.00            16
NH        3

In [71]:
out=make_query("SELECT state,SUM(amount) FROM contributors GROUP BY state;")
make_frame(out, legend=['state','sum'])

   state       sum
0   None   -500.00
1     AK   1210.00
2     AR  14200.00
3     AZ    120.00
4     CA  -5013.73
5     CO  -5823.00
6     CT   2300.00
7     DC  -1549.91
8     FL  -4050.00
9     IA    250.00
10    ID   -261.00
11    IL  -5586.80
12    KS   -330.00
13    KY   -200.00
14    LA   1300.00
15    MA    -83.00
16    MD    300.00
17    ME   2520.00
18    MI  -1265.00
19    MN    322.00
20    MO    100.00
21    NC    500.00
22    NH    -24.60
23    NJ   -817.45
24    NV    725.00
25    NY  -6474.50
26    OH    450.00
27    OK    800.00
28    PA  -2146.00
29    RI    200.00
30    SC   2400.00
31    TN    -25.00
32    TX   1985.24
33    UT   5050.00
34    VA    515.92
35    WA   -500.00

In [72]:
dfcwci.groupby("state")['amount'].mean()

state
AK     403.333333
AR    1183.333333
AZ     120.000000
CA    -217.988261
CO   -1455.750000
CT    2300.000000
DC    -309.982000
FL    -135.000000
IA     250.000000
ID    -261.000000
IL    -931.133333
KS    -330.000000
KY    -200.000000
LA     650.000000
MA     -13.833333
MD     150.000000
ME     630.000000
MI    -253.000000
MN     107.333333
MO     100.000000
NC     500.000000
NH     -24.600000
NJ    -408.725000
NV     181.250000
NY    -809.312500
OH     112.500000
OK     266.666667
PA    -429.200000
RI     100.000000
SC     800.000000
TN     -25.000000
TX     220.582222
UT     459.090909
VA     103.184000
WA    -166.666667
Name: amount, dtype: float64

In [73]:
dfcwci.groupby("state")['amount'].apply(lambda x: np.std(x))

state
AK      81.785628
AR     742.555647
AZ       0.000000
CA     921.394361
CO     887.819907
CT       0.000000
DC     473.728151
FL    1157.594489
IA       0.000000
ID       0.000000
IL    1123.431895
KS       0.000000
KY       0.000000
LA     150.000000
MA     226.572591
MD     100.000000
ME     964.339152
MI    1034.522112
MN     113.611424
MO       0.000000
NC       0.000000
NH       0.000000
NJ     369.225000
NV     184.877493
NY     865.515126
OH     250.935749
OK     169.967317
PA     537.224869
RI       0.000000
SC     509.901951
TN       0.000000
TX     626.481285
UT    1636.316287
VA     209.965120
WA     377.123617
Name: amount, dtype: float64

The dictionary-like structure is more obvious in this method of iteration, but it does not do the combining part...

In [74]:
for k, v in dfcwci.groupby('state'):
    print("State", k, "mean amount", v.amount.mean(), "std", v.amount.std())

State AK mean amount 403.333333333 std 100.166528009
State AR mean amount 1183.33333333 std 775.574078677
State AZ mean amount 120.0 std nan
State CA mean amount -217.98826087 std 942.102437919
State CO mean amount -1455.75 std 1025.16612475
State CT mean amount 2300.0 std nan
State DC mean amount -309.982 std 529.644174536
State FL

 mean amount -135.0 std 1177.38386205
State IA mean amount 250.0 std nan
State ID mean amount -261.0 std nan
State IL mean amount -931.133333333 std 1230.65798119
State KS mean amount -330.0 std nan
State KY mean amount -200.0 std nan
State LA mean amount 650.0 std 212.132034356
State MA mean amount -13.8333333333 std 248.197837756
State MD mean amount 150.0 std 141.421356237
State ME mean amount 630.0 std 1113.52293795
State MI mean amount -253.0 std 1156.63088321
State MN mean amount 107.333333333 std 139.145008295
State MO mean amount 100.0 std nan
State NC mean amount 500.0 std nan
State NH mean amount -24.6 std nan
State NJ mean amount -408.725 std 522.163002567
State NV mean amount 181.25 std 213.478140957
State NY mean amount -809.3125 std 925.274590035
State OH mean amount 112.5 std 289.755644179
State OK mean amount 266.666666667 std 208.166599947
State PA mean amount -429.2 std 600.635663277
State RI mean amount 100.0 std 0.0
State SC mean amount 800.0 std 624.49979984
State 

TX mean amount 220.582222222 std 664.483747615
State UT mean amount 459.090909091 std 1716.18300041
State VA mean amount 103.184 std 234.748140099
State WA mean amount -166.666666667 std 461.880215352


### DELETE

In [75]:
dfcwci.head()

  last_name first_name middle_name                street_1 street_2         city state    zip  amount        date  candidate_id
0      Agee     Steven         NaN  549 Laurel Branch Road      NaN        Floyd    VA  24091   500.0  2007-06-30            16
1    Ahrens        Don         NaN    4034 Rennellwood Way      NaN   Pleasanton    CA  94566   250.0  2007-05-16            16
2    Ahrens        Don         NaN    4034 Rennellwood Way      NaN   Pleasanton    CA  94566    50.0  2007-06-18            16
3    Ahrens        Don         NaN    4034 Rennellwood Way      NaN   Pleasanton    CA  94566   100.0  2007-06-21            16
4      Akin    Charles         NaN  10187 Sugar Creek Road      NaN  Bentonville    AR  72712   100.0  2007-06-16            16

In-place drops

In [76]:
df2=dfcwci.copy()
df2.set_index('last_name', inplace=True)
df2.head()

          first_name middle_name                street_1 street_2         city state    zip  amount        date  candidate_id
last_name                                                                                                                    
Agee          Steven         NaN  549 Laurel Branch Road      NaN        Floyd    VA  24091   500.0  2007-06-30            16
Ahrens           Don         NaN    4034 Rennellwood Way      NaN   Pleasanton    CA  94566   250.0  2007-05-16            16
Ahrens           Don         NaN    4034 Rennellwood Way      NaN   Pleasanton    CA  94566    50.0  2007-06-18            16
Ahrens           Don         NaN    4034 Rennellwood Way      NaN   Pleasanton    CA  94566   100.0  2007-06-21            16
Akin         Charles         NaN  10187 Sugar Creek Road      NaN  Bentonville    AR  72712   100.0  2007-06-16            16

In [77]:
df2.drop(['Ahrens'], inplace=True)
df2.head()

          first_name middle_name                      street_1 street_2         city state    zip  amount        date  candidate_id
last_name                                                                                                                          
Agee          Steven         NaN        549 Laurel Branch Road      NaN        Floyd    VA  24091   500.0  2007-06-30            16
Akin         Charles         NaN        10187 Sugar Creek Road      NaN  Bentonville    AR  72712   100.0  2007-06-16            16
Akin            Mike         NaN              181 Baywood Lane      NaN   Monticello    AR  71655  1500.0  2007-05-18            16
Akin         Rebecca         NaN              181 Baywood Lane      NaN   Monticello    AR  71655   500.0  2007-05-18            16
Aldridge     Brittni         NaN  808 Capitol Square Place, SW      NaN   Washington    DC  20024   250.0  2007-06-06            16

In [78]:
df2.reset_index(inplace=True)
df2.head()

  last_name first_name middle_name                      street_1 street_2         city state    zip  amount        date  candidate_id
0      Agee     Steven         NaN        549 Laurel Branch Road      NaN        Floyd    VA  24091   500.0  2007-06-30            16
1      Akin    Charles         NaN        10187 Sugar Creek Road      NaN  Bentonville    AR  72712   100.0  2007-06-16            16
2      Akin       Mike         NaN              181 Baywood Lane      NaN   Monticello    AR  71655  1500.0  2007-05-18            16
3      Akin    Rebecca         NaN              181 Baywood Lane      NaN   Monticello    AR  71655   500.0  2007-05-18            16
4  Aldridge    Brittni         NaN  808 Capitol Square Place, SW      NaN   Washington    DC  20024   250.0  2007-06-06            16

The recommended way to do it is to create a new dataframe. This might be impractical if things are very large.

In [79]:
dfcwci=dfcwci[dfcwci.last_name!='Ahrens']
dfcwci.head(10)

   last_name first_name middle_name                      street_1 street_2           city state    zip  amount        date  candidate_id
0       Agee     Steven         NaN        549 Laurel Branch Road      NaN          Floyd    VA  24091   500.0  2007-06-30            16
4       Akin    Charles         NaN        10187 Sugar Creek Road      NaN    Bentonville    AR  72712   100.0  2007-06-16            16
5       Akin       Mike         NaN              181 Baywood Lane      NaN     Monticello    AR  71655  1500.0  2007-05-18            16
6       Akin    Rebecca         NaN              181 Baywood Lane      NaN     Monticello    AR  71655   500.0  2007-05-18            16
7   Aldridge    Brittni         NaN  808 Capitol Square Place, SW      NaN     Washington    DC  20024   250.0  2007-06-06            16
8      Allen    John D.         NaN        1052 Cannon Mill Drive      NaN  North Augusta    SC  29860  1000.0  2007-06-11            16
9      Allen    John D.         NaN      

### LIMIT

In [80]:
dfcwci[0:3] # also see loc and iloc from the lab

  last_name first_name middle_name                street_1 street_2         city state    zip  amount        date  candidate_id
0      Agee     Steven         NaN  549 Laurel Branch Road      NaN        Floyd    VA  24091   500.0  2007-06-30            16
4      Akin    Charles         NaN  10187 Sugar Creek Road      NaN  Bentonville    AR  72712   100.0  2007-06-16            16
5      Akin       Mike         NaN        181 Baywood Lane      NaN   Monticello    AR  71655  1500.0  2007-05-18            16

In [81]:
out=make_query("SELECT * FROM contributors LIMIT 3;")
make_frame(out).head(10)

   id last_name first_name middle_name                street_1 street_2        city state    zip  amount        date  candidate_id
0   1      Agee     Steven        None  549 Laurel Branch Road     None       Floyd    VA  24091     500  2007-06-30            16
1   2    Ahrens        Don        None    4034 Rennellwood Way     None  Pleasanton    CA  94566     250  2007-05-16            16
2   3    Ahrens        Don        None    4034 Rennellwood Way     None  Pleasanton    CA  94566      50  2007-06-18            16

### DB's only: indexes

In [82]:
crind="CREATE INDEX amount_ix ON contributors(amount);"
db.cursor().execute(crind)
db.commit()

In [83]:
%%bash
echo ".schema" | sqlite3 /tmp/cancont.db

CREATE TABLE IF NOT EXISTS "candidates" (
    "id" INTEGER PRIMARY KEY  NOT NULL ,
    "first_name" VARCHAR,
    "last_name" VARCHAR,
    "middle_name" VARCHAR,
    "party" VARCHAR NOT NULL
);
CREATE TABLE IF NOT EXISTS "contributors" (
    "id" INTEGER PRIMARY KEY  AUTOINCREMENT  NOT NULL,
    "last_name" VARCHAR,
    "first_name" VARCHAR,
    "middle_name" VARCHAR,
    "street_1" VARCHAR,
    "street_2" VARCHAR,
    "city" VARCHAR,
    "state" VARCHAR,
    "zip" VARCHAR,
    "amount" INTEGER,
    "date" DATETIME,
    "candidate_id" INTEGER NOT NULL,
    FOREIGN KEY(candidate_id) REFERENCES candidates(id)
);
CREATE TABLE sqlite_sequence(name,seq);
CREATE INDEX amount_ix ON contributors(amount);


## Relationships: JOINs are Cartesian Products.

Finally, there are many occasions you will want to combine dataframes. We might want to see who contributed to Obama:

### Simple subselect

In [84]:
dfcand.head()

   id   first_name  last_name middle_name party
0  33       Joseph      Biden         NaN     D
1  36       Samuel  Brownback         NaN     R
2  34      Hillary    Clinton          R.     D
3  39  Christopher       Dodd          J.     D
4  26         John    Edwards         NaN     D

In [85]:
obamaid=dfcand.query("last_name=='Obama'")['id'].values[0]

In [86]:
obamacontrib=dfcwci.query("candidate_id==%i" % obamaid)
obamacontrib.head()

   last_name first_name middle_name         street_1 street_2        city state        zip  amount        date  candidate_id
25   Buckler      Steve         NaN  24351 Armada Dr      NaN  Dana Point    CA  926291306    50.0  2007-07-30            20
26   Buckler      Steve         NaN  24351 Armada Dr      NaN  Dana Point    CA  926291306    25.0  2007-08-16            20
27  Buckheit      Bruce         NaN    8904 KAREN DR      NaN     FAIRFAX    VA  220312731   100.0  2007-09-19            20
28    Buckel      Linda         NaN    PO Box 683130      NaN   Park City    UT  840683130  2300.0  2007-08-14            20
29    Buckel      Linda         NaN    PO Box 683130      NaN   Park City    UT  840683130 -2300.0  2007-08-14            20

In [87]:
russiandollsel="""
SELECT * FROM contributors WHERE 
    candidate_id = (SELECT id from candidates WHERE last_name = 'Obama');
"""
out=make_query(russiandollsel)
make_frame(out).head()

   id last_name first_name middle_name         street_1 street_2        city state        zip  amount        date  candidate_id
0  26   Buckler      Steve        None  24351 Armada Dr     None  Dana Point    CA  926291306    50.0  2007-07-30            20
1  27   Buckler      Steve        None  24351 Armada Dr     None  Dana Point    CA  926291306    25.0  2007-08-16            20
2  28  Buckheit      Bruce        None    8904 KAREN DR     None     FAIRFAX    VA  220312731   100.0  2007-09-19            20
3  29    Buckel      Linda        None    PO Box 683130     None   Park City    UT  840683130  2300.0  2007-08-14            20
4  30    Buckel      Linda        None    PO Box 683130     None   Park City    UT  840683130 -2300.0  2007-08-14            20

### Explicit INNER JOIN

This is the one you will want 90% of the time. It will only match keys that are present in both dataframes.

![inner join](http://pandas.pydata.org/pandas-docs/stable/_images/merging_merge_on_key_inner.png)

(from http://pandas.pydata.org/pandas-docs/stable/merging.html)

In [88]:
cols_wanted=['last_name_x', 'first_name_x', 'candidate_id', 'id', 'last_name_y']
dfcwci.merge(dfcand, left_on="candidate_id", right_on="id")[cols_wanted]

     last_name_x first_name_x  candidate_id  id last_name_y
0           Agee       Steven            16  16    Huckabee
1           Akin      Charles            16  16    Huckabee
2           Akin         Mike            16  16    Huckabee
3           Akin      Rebecca            16  16    Huckabee
4       Aldridge      Brittni            16  16    Huckabee
5          Allen      John D.            16  16    Huckabee
6          Allen      John D.            16  16    Huckabee
7        Allison      John W.            16  16    Huckabee
8        Allison      Rebecca            16  16    Huckabee
9        Allison      Rebecca            16  16    Huckabee
10         Altes         R.D.            16  16    Huckabee
11        Andres         Dale            16  16    Huckabee
12       Anthony         John            16  16    Huckabee
13      Arbogast       Robert            16  16    Huckabee
14      Arbogast       Robert            16  16    Huckabee
15         Ardle      William           

If the names of the columns you wanted to merge on were identical, you could simply say `on=id`, for example, rather than a `left_on` and a `right_on`.

In [89]:
explicitjoinsel="""
SELECT 
    contributors.last_name, contributors.first_name, candidates.last_name 
FROM 
    contributors JOIN candidates 
ON contributors.candidate_id = candidates.id;
"""
out=make_query(explicitjoinsel)
make_frame(out, legend=["contributors.last_name", 
            "contributors.first_name",  "candidates.last_name"]).head()

  contributors.last_name contributors.first_name candidates.last_name
0                   Agee                  Steven             Huckabee
1                 Ahrens                     Don             Huckabee
2                 Ahrens                     Don             Huckabee
3                 Ahrens                     Don             Huckabee
4                   Akin                 Charles             Huckabee

### Outer JOIN

#### left outer (contributors on candidates)

![left outer](http://pandas.pydata.org/pandas-docs/stable/_images/merging_merge_on_key_left.png)

In [90]:
dfcwci.merge(dfcand, left_on="candidate_id", right_on="id", how="left")[cols_wanted]

     last_name_x first_name_x  candidate_id  id last_name_y
0           Agee       Steven            16  16    Huckabee
1           Akin      Charles            16  16    Huckabee
2           Akin         Mike            16  16    Huckabee
3           Akin      Rebecca            16  16    Huckabee
4       Aldridge      Brittni            16  16    Huckabee
5          Allen      John D.            16  16    Huckabee
6          Allen      John D.            16  16    Huckabee
7        Allison      John W.            16  16    Huckabee
8        Allison      Rebecca            16  16    Huckabee
9        Allison      Rebecca            16  16    Huckabee
10         Altes         R.D.            16  16    Huckabee
11        Andres         Dale            16  16    Huckabee
12       Anthony         John            16  16    Huckabee
13      Arbogast       Robert            16  16    Huckabee
14      Arbogast       Robert            16  16    Huckabee
15         Ardle      William           

In [91]:
explicitjoinsel="""
SELECT 
    COUNT(contributors.id), contributors.first_name, candidates.last_name,
        contributors.candidate_id, candidates.id
FROM 
    contributors LEFT OUTER JOIN candidates 
ON contributors.candidate_id = candidates.id

GROUP BY candidates.last_name;
"""
out=make_query(explicitjoinsel)
make_frame(out, legend=["count(contributors.id)", "contributors.first_name",  
            "contributors.candidate_id", "candidates.id", "candidates.last_name"])

   count(contributors.id) contributors.first_name contributors.candidate_id  candidates.id  candidates.last_name
0                      25                     Lin                   Clinton             34                    34
1                      25                 TIMOTHY                  Giuliani             22                    22
2                      25                    John                  Huckabee             16                    16
3                      25                   SALEM                    McCain             37                    37
4                      25                    Ryan                     Obama             20                    20
5                      25                 Richard                      Paul             32                    32
6                      25                  GERALD                    Romney             35                    35

#### right outer (contributors on candidates) = left outer (candidates on contributors)

![right outer](http://pandas.pydata.org/pandas-docs/stable/_images/merging_merge_on_key_right.png)

In [92]:
dfcwci.merge(dfcand, left_on="candidate_id", right_on="id", how="right")[cols_wanted]

     last_name_x first_name_x  candidate_id  id last_name_y
0           Agee       Steven          16.0  16    Huckabee
1           Akin      Charles          16.0  16    Huckabee
2           Akin         Mike          16.0  16    Huckabee
3           Akin      Rebecca          16.0  16    Huckabee
4       Aldridge      Brittni          16.0  16    Huckabee
5          Allen      John D.          16.0  16    Huckabee
6          Allen      John D.          16.0  16    Huckabee
7        Allison      John W.          16.0  16    Huckabee
8        Allison      Rebecca          16.0  16    Huckabee
9        Allison      Rebecca          16.0  16    Huckabee
10         Altes         R.D.          16.0  16    Huckabee
11        Andres         Dale          16.0  16    Huckabee
12       Anthony         John          16.0  16    Huckabee
13      Arbogast       Robert          16.0  16    Huckabee
14      Arbogast       Robert          16.0  16    Huckabee
15         Ardle      William          1

#### full outer

![outer](http://pandas.pydata.org/pandas-docs/stable/_images/merging_merge_on_key_outer.png)

In [93]:
dfcwci.merge(dfcand, left_on="candidate_id", right_on="id", how="outer")[cols_wanted]

     last_name_x first_name_x  candidate_id  id last_name_y
0           Agee       Steven          16.0  16    Huckabee
1           Akin      Charles          16.0  16    Huckabee
2           Akin         Mike          16.0  16    Huckabee
3           Akin      Rebecca          16.0  16    Huckabee
4       Aldridge      Brittni          16.0  16    Huckabee
5          Allen      John D.          16.0  16    Huckabee
6          Allen      John D.          16.0  16    Huckabee
7        Allison      John W.          16.0  16    Huckabee
8        Allison      Rebecca          16.0  16    Huckabee
9        Allison      Rebecca          16.0  16    Huckabee
10         Altes         R.D.          16.0  16    Huckabee
11        Andres         Dale          16.0  16    Huckabee
12       Anthony         John          16.0  16    Huckabee
13      Arbogast       Robert          16.0  16    Huckabee
14      Arbogast       Robert          16.0  16    Huckabee
15         Ardle      William          1

When to use which?

See this:

http://blog.codinghorror.com/a-visual-explanation-of-sql-joins/

## Pandas /SQL

In [94]:
pd.read_sql("SELECT * FROM candidates WHERE party= 'D';", db)

   id   first_name   last_name middle_name party
0  20       Barack       Obama        None     D
1  24         Mike      Gravel        None     D
2  26         John     Edwards        None     D
3  29         Bill  Richardson        None     D
4  31       Dennis    Kucinich        None     D
5  33       Joseph       Biden        None     D
6  34      Hillary     Clinton          R.     D
7  39  Christopher        Dodd          J.     D

### Implicit Join

In [95]:
implicitjoinsel="""
SELECT 
    contributors.last_name, contributors.first_name, contributors.amount, candidates.last_name 
FROM 
    contributors, candidates 
WHERE contributors.candidate_id = candidates.id
AND candidates.last_name = 'Obama';
"""
out=make_query(implicitjoinsel)
make_frame(out, legend=["contributors.last_name", 
            "contributors.first_name", "contributors.amount", "candidates.last_name"]).head()

  contributors.last_name contributors.first_name  contributors.amount candidates.last_name
0                Buckler                   Steve                 50.0                Obama
1                Buckler                   Steve                 25.0                Obama
2               Buckheit                   Bruce                100.0                Obama
3                 Buckel                   Linda               2300.0                Obama
4                 Buckel                   Linda              -2300.0                Obama

In [96]:
pd.read_sql(implicitjoinsel, db)

   last_name first_name   amount last_name
0    Buckler      Steve    50.00     Obama
1    Buckler      Steve    25.00     Obama
2   Buckheit      Bruce   100.00     Obama
3     Buckel      Linda  2300.00     Obama
4     Buckel      Linda -2300.00     Obama
5     Buckel      Linda  4600.00     Obama
6       Buck     Thomas   100.00     Obama
7       Buck        Jay   200.00     Obama
8       Buck     Blaine  2300.00     Obama
9       Buck    Barbara    50.00     Obama
10      Buck    Barbara    50.00     Obama
11   Buchman     Mark M   460.80     Obama
12    Bucher        Ida   100.00     Obama
13  Buchanek  Elizabeth    50.00     Obama
14  Buchanan       John   500.00     Obama
15  Buchanan       John  -500.00     Obama
16  Buchanan       John   500.00     Obama
17  Buchanan       John   700.00     Obama
18  Buchanan       John  -700.00     Obama
19  Buchanan       John  1000.00     Obama
20  Buchanan       John  1300.00     Obama
21  Buchanan       John   200.00     Obama
22  Buchana

## Useful Links

- http://sebastianraschka.com/Articles/sqlite3_database.html and  http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html#unique_indexes
- https://github.com/tthibo/SQL-Tutorial
- http://chrisalbon.com

And especially for R users:

- https://cran.r-project.org/web/packages/dplyr/vignettes/introduction.html
- https://gist.github.com/TomAugspurger/6e052140eaa5fdb6e8c0/